# ASL Action Recognition

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/03-emotion-analysis.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/03-emotion-analysis.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/03-emotion-analysis.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Start EVA server
We are reusing the start server notebook for launching the EVA server

In [8]:
#!wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

nohup eva_server > eva.log 2>&1 &
Note: you may need to restart the kernel to use updated packages.


In [11]:
# An ASL video of the action computer
!wget -nc https://gatech.box.com/shared/static/ktdkbu55tp4c0itdb3q629ajbv99po8b.mp4 -O computer_asl.mp4 

#ASL UDF
!wget -nc https://raw.githubusercontent.com/georgia-tech-db/eva/master/eva/udfs/asl_action_recognition.py

#Pickle file containing the mapping of the ASL action to index
!wget -nc https://raw.githubusercontent.com/georgia-tech-db/eva/master/eva/udfs/asl_20_actions_map.pkl


--2023-01-19 00:02:57--  https://gatech.box.com/shared/static/ktdkbu55tp4c0itdb3q629ajbv99po8b.mp4
Resolving gatech.box.com (gatech.box.com)... 74.112.186.144
Connecting to gatech.box.com (gatech.box.com)|74.112.186.144|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/ktdkbu55tp4c0itdb3q629ajbv99po8b.mp4 [following]
--2023-01-19 00:02:58--  https://gatech.box.com/public/static/ktdkbu55tp4c0itdb3q629ajbv99po8b.mp4
Reusing existing connection to gatech.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gatech.app.box.com/public/static/ktdkbu55tp4c0itdb3q629ajbv99po8b.mp4 [following]
--2023-01-19 00:02:58--  https://gatech.app.box.com/public/static/ktdkbu55tp4c0itdb3q629ajbv99po8b.mp4
Resolving gatech.app.box.com (gatech.app.box.com)... 74.112.186.144
Connecting to gatech.app.box.com (gatech.app.box.com)|74.112.186.144|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loca

### Adding the video file to EVADB for analysis

In [12]:
cursor.execute('DROP TABLE Asl_actions')
response = cursor.fetch_all()
print(response)
cursor.execute('LOAD VIDEO "computer_asl.mp4" INTO Asl_actions')
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                          0
0  Table Successfully dropped: Asl_actions
@query_time: 0.027660994324833155
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded VIDEO: 1
@query_time: 0.06865505781024694


### Visualize the video

In [13]:
from IPython.display import Video
Video("computer_asl.mp4", height=450, width=800, embed=True)

### Create an user-defined function(UDF) for analyzing the frames

In [14]:
cursor.execute("""CREATE UDF IF NOT EXISTS ASLActionRecognition
        INPUT  (Frame_Array NDARRAY UINT8(3, 16, 224, 224))
        OUTPUT (labels NDARRAY STR(ANYDIM))
        TYPE  Classification
        IMPL  'asl_action_recognition.py'
        """)
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                                          0
0  UDF ASLActionRecognition already exists, nothing added.
@query_time: 0.015041488222777843


### Run the UDF on the Video to find the ASL action

In [15]:
cursor.execute("""SELECT FIRST(id), ASLActionRecognition(SEGMENT(data)) FROM Asl_actions
                        SAMPLE 5 GROUP BY '16f';""")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
    asl_actions.id aslactionrecognition.labels
0               0                    computer
@query_time: 2.089962868951261
